In [4]:
import sys
import rospy
from sensor_msgs.msg import Image
import cv2
from cv_bridge import CvBridge
cv_bridge = CvBridge()
def image_cb(img_msg):
    # Transform to cv2/numpy image
    img_in_cv2 = cv_bridge.imgmsg_to_cv2(
        img_msg, desired_encoding='passthrough')
    # Transform to grayscale,
    # available encodings: http://docs.ros.org/jade/api/sensor_msgs/html/image__encodings_8h_source.html
    if "rgb" in img_msg.encoding:
        gray_img = cv2.cvtColor(img_in_cv2, cv2.COLOR_RGB2GRAY)
    elif "bgr" in img_msg.encoding:
        gray_img = cv2.cvtColor(img_in_cv2, cv2.COLOR_BGR2GRAY)
    # Transform back to Image message
    gray_img_msg = cv_bridge.cv2_to_imgmsg(
        gray_img, encoding="mono8")
    return gray_img_msg


In [5]:
from rosbag import Bag
import std_msgs.msg
seq = 0
seqC = 0
seqI = 0
with Bag('UAV.bag', 'w') as Y:
    for topic, msg, t in Bag('2022-01-20-00-09-47.bag'):
        if topic == '/camera/color/image_raw':
            msg = image_cb(msg)
            msg.header.stamp = t
            msg.header.frame_id = 'camera'
            msg.header.seq = seqC
            seqC += 1
            Y.write('/sensor_platform/camera_visual/img', msg, t)
        elif topic == '/imu/data':
            msg.header.stamp = t
            msg.header.seq = seqI
            seqI += 1
            msg.header.frame_id = 'base_link'
            Y.write('/sensor_platform/imu', msg, t)
        elif topic == '/ti_mmwave/radar_scan_pcl_0':
            h = std_msgs.msg.Header()
            h.stamp = t
            h.seq = seq
            h.frame_id = 'radar'
            seq += 1
            msg.header = h
            Y.write('/sensor_platform/radar/scan', msg, t)
            Y.write('/sensor_platform/radar/trigger', h, t)

In [6]:
from rosbag import Bag
import std_msgs.msg
for topic, msg, t in Bag('UAV.bag'):
    if topic == '/sensor_platform/radar/scan' :
        print(msg.fields)
        break


[name: "x"
offset: 0
datatype: 7
count: 1, name: "y"
offset: 4
datatype: 7
count: 1, name: "z"
offset: 8
datatype: 7
count: 1, name: "intensity"
offset: 16
datatype: 7
count: 1, name: "velocity"
offset: 20
datatype: 7
count: 1]
